# Find all 2-to-1 purification protocols in 1G1B

Here, we find the output fidelity and success probability of every bilocal Clifford protocols for 2-to-1 purification using two diagonal Bell states as input.

In [34]:
import numpy as np
import itertools as it
import matplotlib.pyplot as plt
from scipy.io import savemat

## Functions

In [35]:
def base(M, n):
    # calculate the image of the base under a matrix M
    
    # make a set of all combinations of the first column and the last n columns (these correspond to X_1, Z_1,...,Z_n)
    s = []
    for i in range(n+1, 2*n):
        s.append(M[0:2*n, i])
    powerset = it.chain.from_iterable(it.combinations(s, r) for r in range(1, len(s)+1))
    
    res = [vector(GF(2),2*n)]
        
    for i in powerset:
        v = vector(sum(i))     # calculate the sum of the elements of each combination (e.g IZZ = IZI + IIZ)
        res.append(v)
        
    return res

In [36]:
def pillars(M, n):
    # calculate the image of the pillars under a matrix M
    
    X1 = vector(M[0:2*n, 0])
    Z1 = vector(M[0:2*n, n])
    Y1 = X1 + Z1
    
    pI = base(M, n)
    pX = [(X1 + b) for b in pI]
    pY = [(Y1 + b) for b in pI]
    pZ = [(Z1 + b) for b in pI]
    
    return [pI, pX, pY, pZ]   

In [37]:
def tensor(A, n):
    # calculate the n fold tensor product of a matrix A
    
    kron = A
    count = 1
    while count < n:
        kron = np.kron(kron,A)
        count = count + 1
        
    if n == 2:
        res = np.reshape(kron, (4,4))
    elif n == 3:
        res = np.reshape(kron, (4,4,4)) 
    elif n == 4:
        res = np.reshape(kron, (4,4,4,4)) 
    elif n == 5:
        res = np.reshape(kron, (4,4,4,4,4)) 
        
    return res

In [38]:
def dist_stat(initial, M, n):
    # returns the success probability and the fidelity of an n-to-1 protocol M applied to an initial state
    pil = pillars(M, n)
    out = []
    for layer in pil:   
        coef = 0
        for elt in layer:
            if n == 2:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1])]
            if n == 3:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1]), \
                                   int(elt[2]) + 2*int(elt[n+2])]
            if n == 4:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1]), \
                                   int(elt[2]) + 2*int(elt[n+2]), int(elt[3]) + 2*int(elt[n+3])]
            if n == 5:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1]), \
                                   int(elt[2]) + 2*int(elt[n+2]), int(elt[3]) + 2*int(elt[n+3]), \
                                    int(elt[4]) + 2*int(elt[n+4])]
        out.append(coef)
    sp = sum(out)
    fid = out[0]/sp

    return round(sp,10), round(fid,10)

In [39]:
def sucprob_fid_lists(initial, transversal_inv, n):
    # calculate the possible distillation statistics (success probability & fidelity) of the protocols in a transversal
    # applied to an initial state
          
    fid = []
    sp = []
    fslist = set()
    for key, M in transversal_inv.items():
        s, f = dist_stat(initial, M, n)
        if (s,f) not in fslist:
            sp.append(s)
            fid.append(f)
            fslist.add((s,f))

    return sp, fid

## Inputs

In [40]:
m = 2 # Number of qubits
F_min = 0.25 # Minimum fidelity for any memory
F_max = 1 # Maximum fidelity for any memory
dF = 0.01 # Step size in fidelity calculation

## Calculations

In [41]:
# Load transversal
transversal_inv = load('2_transversal_inv.sobj')

In [42]:
## If we want a state in Bell diagonal basis we do it like this:
# F = 0.75
# g1 = 0.15
# g2 = 0.1
# g3 = 1 - F - g1 - g2
# assert g3>=0
# state = np.diag([F,g1,g2,g3])

In [18]:
# WERNER STATE

data = dict()

for F_good in np.arange(F_min,F_max+dF/2,dF):
    F_good = round(F_good,2)
    for F_bad in np.arange(F_min,F_max+dF/2,dF):
        # No need to loop over all possible F_bad due to symmetry
        # in the inputs to the purification circuit
        F_bad = round(F_bad,2)

        # Good memory with Werner state
        g = (1-F_good)/3
        assert g>=0
        good_state = np.diag([F_good,g,g,g])

        # Bad memory with Werner state
        b = (1-F_bad)/3
        assert b>=0
        bad_state = np.diag([F_bad,b,b,b])
        
        # Tensor product
        init = np.matmul(bad_state,good_state) # Diagonal entries
        for i in range(len(init)):
            for j in range(len(init)):
                init[i][j] = good_state[i][i] * bad_state[j][j]
                
        # Solve
        probs, fids = sucprob_fid_lists(init, transversal_inv, m)

        # Save
        fid_bad = '%.3f'%(F_bad/10)
        fid_good = '%.3f'%(F_good/10)
        variable_name = 'protocols_F%s_F%s'%(fid_bad[2:],fid_good[2:])
        data[variable_name] = np.array([probs, fids])

savemat("data_protocols_Werner.mat", data)

In [19]:
# 3-2-1 BELL DIAGONAL STATE

data = dict()

for F_good in np.arange(F_min,F_max+dF/2,dF):
    F_good = round(F_good,2)
    for F_bad in np.arange(F_min,F_max+dF/2,dF):
        # No need to loop over all possible F_bad due to symmetry
        # in the inputs to the purification circuit
        F_bad = round(F_bad,2)

        # Good memory with Bell diagonal state
        g1 = 3*(1-F_good)/6
        g2 = 2*(1-F_good)/6
        g3 = 1-F_good-g1-g2
        assert g3>=0
        good_state = np.diag([F_good,g1,g2,g3])

        # Bad memory with Bell diagonal state
        b1 = 3*(1-F_bad)/6
        b2 = 2*(1-F_bad)/6
        b3 = 1-F_bad-b1-b2
        assert b3>=0
        bad_state = np.diag([F_bad,b1,b2,b3])
        
        # Tensor product
        init = np.matmul(bad_state,good_state) # Diagonal entries
        for i in range(len(init)):
            for j in range(len(init)):
                init[i][j] = good_state[i][i] * bad_state[j][j]
                
        # Solve
        probs, fids = sucprob_fid_lists(init, transversal_inv, m)

        # Save
        fid_bad = '%.3f'%(F_bad/10)
        fid_good = '%.3f'%(F_good/10)
        variable_name = 'protocols_F%s_F%s'%(fid_bad[2:],fid_good[2:])
        data[variable_name] = np.array([probs, fids])

savemat("data_protocols_321.mat", data)

In [33]:
# WERNER AND R-STATE

data = dict()

for F_good in np.arange(F_min,F_max+dF/2,dF):
    F_good = round(F_good,2)
    for F_bad in np.arange(F_min,F_max+dF/2,dF):
        # No need to loop over all possible F_bad due to symmetry
        # in the inputs to the purification circuit
        F_bad = round(F_bad,2)

        # Good memory with Werner state
        g = (1-F_good)/3
        assert g>=0
        good_state = np.diag([F_good,g,g,g])

        # Bad memory with (twirled) R-state
        b1 = (1-F_bad)/2
        b2 = (1-F_bad)/2
        b3 = 0
        assert b3>=0
        bad_state = np.diag([F_bad,b1,b2,b3])
        
        # Tensor product
        init = np.matmul(bad_state,good_state) # Diagonal entries
        for i in range(len(init)):
            for j in range(len(init)):
                init[i][j] = good_state[i][i] * bad_state[j][j]
                
        # Solve
        probs, fids = sucprob_fid_lists(init, transversal_inv, m)

        # Save
        fid_bad = '%.3f'%(F_bad/10)
        fid_good = '%.3f'%(F_good/10)
        variable_name = 'protocols_F%s_F%s'%(fid_bad[2:],fid_good[2:])
        data[variable_name] = np.array([probs, fids])

savemat("data_protocols_WernerR.mat", data)

In [43]:
# BELL DIAGONAL EXAMPLE

data = dict()

for F_good in np.arange(F_min,F_max+dF/2,dF):
    F_good = round(F_good,2)
    for F_bad in np.arange(F_min,F_max+dF/2,dF):
        # No need to loop over all possible F_bad due to symmetry
        # in the inputs to the purification circuit
        F_bad = round(F_bad,2)

        # Good memory
        good_state = np.diag([F_good,0,0,1-F_good])

        # Bad memory
        bad_state = np.diag([F_bad,0,0,1-F_bad])
        
        # Tensor product
        init = np.matmul(bad_state,good_state) # Diagonal entries
        for i in range(len(init)):
            for j in range(len(init)):
                init[i][j] = good_state[i][i] * bad_state[j][j]
                
        # Solve
        probs, fids = sucprob_fid_lists(init, transversal_inv, m)

        # Save
        fid_bad = '%.3f'%(F_bad/10)
        fid_good = '%.3f'%(F_good/10)
        variable_name = 'protocols_F%s_F%s'%(fid_bad[2:],fid_good[2:])
        data[variable_name] = np.array([probs, fids])

savemat("data_protocols_example.mat", data)